In [3]:
!pip install dlib
!pip install opencv-python
!pip install cmake
!pip install face_recognition

In [4]:
import cv2
import face_recognition

# Inicializa variável com nome do usuário a ser identificado
NOME = 'Vinicius'

# Carrega uma imagem de exemplo do seu rosto
known_image = face_recognition.load_image_file("me.jpg")
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# Inicializa algumas variáveis
face_locations = []
face_encodings = []
process_this_frame = True
text_to_display = ""
last_known_face_found = False

# Inicializa a webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Captura quadro a quadro do vídeo
    ret, frame = video_capture.read()

    # Converte a imagem de cor BGR (OpenCV) para RGB (face_recognition)
    rgb_frame = frame[:, :, ::-1]

    # Redimensiona o quadro para 1/4 do tamanho para um processamento mais rápido de reconhecimento facial
    small_rgb_frame = cv2.resize(rgb_frame, (0, 0), fx=0.25, fy=0.25)

    # Processa apenas cada segundo quadro de vídeo para economizar tempo
    if process_this_frame:
        # Encontra todos os rostos e codificações de rostos no quadro atual do vídeo
        face_locations = face_recognition.face_locations(small_rgb_frame)
        if face_locations:  # Verifica se algum rosto foi encontrado
            face_encodings = face_recognition.face_encodings(small_rgb_frame, face_locations)

            # Verifica se algum rosto corresponde ao rosto conhecido
            if face_encodings:  # Verifica se face_encodings não está vazio
                for face_encoding in face_encodings:
                    # Compara a codificação de rosto do rosto atual com a codificação de rosto conhecido
                    results = face_recognition.compare_faces([known_face_encoding], face_encoding, tolerance=0.6)
                    if results[0]:
                        if not last_known_face_found:
                            text_to_display = f"Rosto do usuario {NOME} identificado!"
                            last_known_face_found = True
                    else:
                        text_to_display = "Rosto desconhecido"
                        last_known_face_found = False
        else:
            text_to_display = "Nenhum rosto encontrado"
            last_known_face_found = False

    # Exibe o texto no quadro de vídeo
    if text_to_display:
        cv2.putText(frame, text_to_display, (15, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Exibe a imagem resultante
    cv2.imshow('Identificador Facial', frame)

    # Pressione 'q' no teclado para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    process_this_frame = not process_this_frame

# Libera o acesso à webcam
video_capture.release()
cv2.destroyAllWindows()